In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
import os
def getHistory(season, current_season):
    gw = pd.DataFrame()
    PATH = "C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/" + season + "/gws/"
    files = []
    for r, d, f in os.walk(PATH):
        for file in f:
            if '.csv' in file:
                if(file != 'merged_gw.csv'):
                    files.append(os.path.join(r, file))

    for f in files:
        temp = pd.read_csv(f, encoding="latin-1")    
        temp['gw'] = f.split('/')[-1].split('.')[0][2:]
        temp['season'] = season 
        gw = gw.append(temp, ignore_index=True)
    
    if len(gw) > 0:
        gw['kickoff_time'] = pd.to_datetime(gw['kickoff_time'])
        gw['date'] = pd.to_datetime(gw['kickoff_time']).dt.date
    else:
        return gw

    if int(season.split('-')[0]) > 2018:
        teams = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + season + '/teams.csv', encoding="latin-1")
        teams = teams[['id', 'short_name']]
        gw = pd.merge(gw, teams, 'left', left_on='opponent_team', right_on = 'id')
        gw.drop(['opponent_team'], axis = 1, inplace = True)
        gw.rename(columns = {'short_name': 'opponent_team_name'}, inplace = True)
        
        players = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + season + '/players_raw.csv', encoding="latin-1")        
        squad = players[['first_name', 'second_name', 'element_type', 'id', 'team', 'now_cost', 'code']]
        gw = pd.merge(gw, squad, 'left', left_on = "element", right_on='id')
        if(int(season.split('-')[0]) == 2020):
            gw.drop(['team_x'], axis = 1, inplace =  True)
            gw = gw.rename(columns = {'team_y': 'team'})
        gw = pd.merge(gw, teams, 'left', left_on = 'team', right_on ='id')
        gw.drop(['team', 'id_y'], axis = 1, inplace =  True)
        gw = gw.rename(columns = {'short_name': 'team_name',
                                  'id_x': 'id'})
    else:
        teams = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/Personal/Projects/FPL/teams' + season.replace('-', '') + '.csv', encoding="latin-1")
        players = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + season + '/players_raw.csv', encoding="latin-1")
        squad = players[['first_name', 'second_name', 'element_type', 'id', 'team', 'code']]
        gw = pd.merge(gw, teams, 'left', on ='opponent_team')
        gw = pd.merge(gw, squad, 'left', left_on = "element", right_on='id')
        gw = pd.merge(gw, teams, 'left', left_on = 'team', right_on ='opponent_team')
        gw.drop(['opponent_team_y', 'id_y'], axis = 1, inplace =  True)
        gw = gw.rename(columns = {'opponent_team_name_x': 'opponent_team_name', 
                                  'opponent_team_name_y': 'team_name',
                                  'id_x': 'id'})
    
    di = {1: "GK", 2: "DEF", 3: "MID", 4: "FWD"}
    if season == current_season:
        gw['new_position'] = gw['element_type'].map(di)
        gw.drop('element_type', axis = 1, inplace = True)
        gw['name'] = gw['first_name'] +  ' ' + gw['second_name']
        gw = gw[~(gw['minutes'] == 0)]
    else:        
        gw['old_position'] = gw['element_type'].map(di)
        gw.drop('element_type', axis = 1, inplace = True)
        
        players_current_season = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + current_season + '/players_raw.csv', encoding='latin-1')
        squad_current_season = players_current_season[['first_name', 'second_name', 'element_type', 'now_cost', 'code']]
        gw = pd.merge(gw, squad_current_season, 'left', on = 'code', suffixes = ('', '_y'))
        gw['new_position'] = gw['element_type'].map(di)
        gw.drop(['element_type', 'first_name_y', 'second_name_y'], axis = 1, inplace = True)
        gw['name'] = gw['first_name'] +  ' ' + gw['second_name']
        
        if gw.columns.contains('now_cost_y'):
            gw.drop('now_cost', axis = 1, inplace = True)
            gw = gw.rename(columns = {'now_cost_y': 'now_cost'})
        gw = gw[~(gw['now_cost'].isnull())]
        gw = gw[~(gw['minutes'] == 0)]
        
        condition = (gw.old_position == 'MID') & (gw.new_position == 'FWD')
        gw.loc[condition, 'total_points'] = gw.loc[condition, 'total_points'] - gw.loc[condition, 'goals_scored'] - gw.loc[condition, 'clean_sheets']

        condition = (gw.old_position == 'FWD') & (gw.new_position == 'MID')
        gw.loc[condition, 'total_points'] = gw.loc[condition, 'total_points'] + gw.loc[condition, 'goals_scored'] + gw.loc[condition, 'clean_sheets']

        condition = (gw.old_position == 'MID') & (gw.new_position == 'DEF')
        gw.loc[condition, 'total_points'] = gw.loc[condition, 'total_points'] + gw.loc[condition, 'goals_scored'] + 4*gw.loc[condition, 'clean_sheets']

        condition = (gw.old_position == 'DEF') & (gw.new_position == 'MID')
        gw.loc[condition, 'total_points'] = gw.loc[condition, 'total_points'] - gw.loc[condition, 'goals_scored'] - 4*gw.loc[condition, 'clean_sheets']
    
    gw = gw[~(gw['kickoff_time'].isnull())]
    gw = gw[['kickoff_time', 'season', 'gw', 'name', 'team_name', 'opponent_team_name', 'team_h_score', 'team_a_score', 'new_position', 'total_points', 'minutes', 'bonus', 
             'goals_scored', 'goals_conceded', 'assists', 'was_home', 'yellow_cards', 'red_cards', 'saves', 
             'threat', 'ict_index', 'clean_sheets', 'now_cost', 'code']]
    return gw

In [3]:
# gw2016 = getHistory(season = "2016-17", current_season = "2019-20")
# gw2017 = getHistory(season = "2017-18", current_season = "2019-20")
# gw2018 = getHistory(season = "2018-19", current_season = "2019-20")
# gw2019 = getHistory(season = "2019-20", current_season = "2019-20")

In [4]:
gw2016 = getHistory(season = "2016-17", current_season = "2020-21")
gw2017 = getHistory(season = "2017-18", current_season = "2020-21")
gw2018 = getHistory(season = "2018-19", current_season = "2020-21")
gw2019 = getHistory(season = "2019-20", current_season = "2020-21")
gw2020 = getHistory(season = "2020-21", current_season = "2020-21")

In [5]:
gw2016.shape, gw2017.shape, gw2018.shape, gw2019.shape, gw2020.shape

((4428, 24), (5840, 24), (7403, 24), (8962, 24), (4234, 24))

In [6]:
gw = pd.concat((gw2016, gw2017, gw2018, gw2019, gw2020))
gw['gw'] = gw['gw'].astype('int')
gw = gw.sort_values(['season', 'gw'], ascending = True)
gw_original = gw

In [7]:
gw_original.to_csv('GW.csv', index=False)

In [8]:
def points_adj_exp_decay_2(group, multiply_factor):
    group = group.sort_values(['season', 'gw'], ascending = [False, False]).reset_index(drop = True)
    N = 1
    factor = []
    for i in range(0, len(group)):
        #N = np.exp(np.log(N) + (-i * multiply_factor))
        #N = np.exp(np.log(N) - np.exp(np.log(i) + np.log(multiply_factor)))
        N = N * np.exp(-i * multiply_factor)
        factor.append(N)
    group['total_points_adj'] = group['total_points'] * factor
    group['minutes_adj'] = group['minutes'] * factor
    
#     group['total_points_adj'] = (group['total_points'].apply(lambda x: np.log(x)) + 
#                                  [np.log(x) for x in factor]).apply(lambda x: np.exp(x))
#     group['minutes_adj'] = (group['minutes'].apply(lambda x: np.log(x)) + 
#                             [np.log(x) for x in factor]).apply(lambda x: np.exp(x))
    
    group['ppm90'] = ((group.total_points_adj/group.minutes_adj)*90)
    group['ppm90_orig'] = ((group.total_points/group.minutes)*90)
    return group

In [9]:
def fixture_decay(group, multiply_factor):
    group = group.sort_values(['season', 'gw'], ascending = [False, False]).reset_index(drop = True)
    N = 1
    factor = []
    for i in range(0, len(group)):
        N = N * np.exp(-i*multiply_factor)
        factor.append(N)
    group['team_h_score_adj'] = group['team_h_score'] * factor
    group['team_a_score_adj'] = group['team_a_score'] * factor
    group['minutes_adj'] = group['minutes'] * factor
    return group

In [10]:
#Initialize weights
def getPPM(player_points_decay, 
           fixture_difficulty_decay, 
           fixture_clusters, 
           max_gw, 
           next_fixture_count, 
           season_not_started, 
           max_of_next_gws,
           lookahead,
           skip_next_gw,
           player_inclusions):
    gw = gw_original
    current_season = str(gw.season.str.split('-').str[0].astype('int').drop_duplicates().nlargest()[0:1].values[0])
    #max_gw = gw[(gw.season.str.split('-').str[0] == current_season)].gw.max()
    gw = gw[((gw.season.str.split('-').str[0] == current_season) & (gw['gw'] <= max_gw)) | (gw.season.str.split('-').str[0] != current_season)]
    last_gw_kickoff = gw[(gw.season.str.split('-').str[0] == current_season) & (gw['gw'] == max_gw)].kickoff_time.max()
    gw['total_points'] = gw['total_points'].astype('float64')
    gw['minutes'] = gw['minutes'].astype('float64')
       
    #Adjust points with an exponential decay function
    gw = gw.groupby('name').apply(lambda x: points_adj_exp_decay_2(x, player_points_decay)).reset_index(drop = True)
    
#     print((gw[gw.name.str.contains('Vardy')].sum()['total_points']/gw[gw.name.str.contains('Vardy')].sum()['minutes'])*90)
#     print((gw[gw.name.str.contains('Vardy')].sum()['total_points_adj']/gw[gw.name.str.contains('Vardy')].sum()['minutes_adj'])*90)
#     print(gw[gw.name.str.contains('Vardy')].sum()['ppm90_orig'])
#     print(gw[gw.name.str.contains('Vardy')].sum()['ppm90'])
    
    #Get Fixture Difficulty
    fixture_difficulty = gw.drop_duplicates(['season', 'team_name', 'opponent_team_name', 'was_home'])
    fixture_difficulty = fixture_difficulty[['season', 'gw', 'team_name', 'opponent_team_name', 
                                             'team_h_score', 'team_a_score', 'was_home']]
    fixture_difficulty['minutes'] = 90
   
    fixture_difficulty = fixture_difficulty.groupby('team_name').apply(lambda x: fixture_decay(x, fixture_difficulty_decay)).reset_index(drop = True)

    fixture_difficulty.loc[:,'goals_scored_adj'] = np.where(fixture_difficulty['was_home'] == True, 
                                                               fixture_difficulty['team_h_score_adj'],
                                                               fixture_difficulty['team_a_score_adj'])

    fixture_difficulty.loc[:,'goals_conceded_adj'] = np.where(fixture_difficulty['was_home'] == False, 
                                                               fixture_difficulty['team_h_score_adj'],
                                                               fixture_difficulty['team_a_score_adj'])

    fixture_difficulty = fixture_difficulty.groupby(['team_name', 'was_home']).agg({'goals_scored_adj': 'sum',
                                                                                  'goals_conceded_adj': 'sum',
                                                                                  'minutes_adj': 'sum'}).reset_index()

    fixture_difficulty['goals_scored_adj_per_minute'] = fixture_difficulty['goals_scored_adj'] / fixture_difficulty['minutes_adj']
    fixture_difficulty['goals_conceded_adj_per_minute'] = fixture_difficulty['goals_conceded_adj'] / fixture_difficulty['minutes_adj']

    fixture_difficulty = fixture_difficulty[['team_name', 'was_home', 'goals_scored_adj_per_minute', 'goals_conceded_adj_per_minute']]

    data = pd.get_dummies(fixture_difficulty, columns = ['was_home'], drop_first = True)
    X = data[['goals_scored_adj_per_minute', 'goals_conceded_adj_per_minute', 'was_home_True']].values
    
    from sklearn.cluster import KMeans
    def get_clusters(X, nClusters):
        kmeans = KMeans(n_clusters = nClusters, random_state=0).fit(X)
        return kmeans.labels_

    fixture_difficulty['cluster'] = get_clusters(X, fixture_clusters)
    fixture_difficulty.rename(columns={'cluster': 'fixture_difficulty'}, inplace=True)
    gw['opposition_was_home'] = (gw['was_home'] == False)
    
    gw = pd.merge(gw, fixture_difficulty, left_on = ['opponent_team_name', 'opposition_was_home'], right_on = ['team_name', 'was_home'])
    
    gw.rename(columns={'team_name_x': 'team_name'}, inplace=True)
    gw.drop('team_name_y', axis=1,inplace=True)

    most_points = gw.groupby(['season', 'team_name', 'new_position', 'name', 'fixture_difficulty']).agg({
                                                                    'total_points': 'sum', 
                                                                    'total_points_adj': 'sum', 
                                                                    'minutes': 'sum', 
                                                                    'minutes_adj': 'sum', 
                                                                    'bonus': 'sum',
                                                                    'goals_scored': 'sum',
                                                                    'assists': 'sum',
                                                                    'threat': 'sum',
                                                                    'ict_index': 'sum',                                                                
                                                                    'clean_sheets': 'sum',
                                                                    'now_cost': 'max'
                                                                    }).sort_values('total_points_adj', ascending = False).reset_index()

    print(most_points.head(10))
    
    most_points['90count'] = (most_points['minutes']/90).round().astype('int')
    most_points['ppm90'] = ((most_points.total_points_adj/most_points.minutes_adj)*90)
    most_points['ppm90_orig'] = ((most_points.total_points/most_points.minutes)*90)
    #print(most_points[most_points.name.str.contains('Bruyne')])

    overall_games = most_points.groupby(['team_name', 'new_position', 'name']).agg({'minutes': 'sum'}).reset_index()
    overall_games['overall_90count'] = (overall_games['minutes']/90).round().astype('int')
    overall_games = overall_games.drop('minutes', axis=1)

    most_points = pd.merge(most_points, overall_games, on = ['team_name', 'new_position', 'name'])

    season_games = most_points.groupby(['season', 'team_name', 'new_position', 'name']).agg({'minutes': 'sum'}).reset_index()
    season_games['season_90count'] = (season_games['minutes']/90).round().astype('int')
    season_games = season_games.drop('minutes', axis=1)
    
    gw['season_gw'] = gw.season.str.split('-').str[0] + '-' + gw.gw.astype('str').str.zfill(2)
    last_4gws = gw.season_gw.sort_values().unique()[-3:]
    print(last_4gws)
    last_4gws_data = gw[gw['season_gw'].isin(last_4gws)]
    last_4gws_min = last_4gws_data.groupby('name').sum()['minutes'].reset_index()
    print(last_4gws_min[last_4gws_min.name.str.contains('Bruyne')])
    last_4gws_min = last_4gws_min[last_4gws_min['minutes'] >= 180].reset_index()

    most_points = pd.merge(most_points, season_games, on = ['season', 'team_name', 'new_position', 'name'])
    
    most_points = most_points[(most_points.name.isin(

                    most_points[
                   ((most_points.season.str.split('-').str[0] == str(most_points.season.str.split('-').str[0].astype('int')\
                                                              .drop_duplicates().nlargest()[0:1].values[0]))
                      & (most_points['season_90count'] >= max_gw/2))

                   &(((most_points.season.str.split('-').str[0] == str(most_points.season.str.split('-').str[0].astype('int')\
                                                              .drop_duplicates().nlargest()[1:2].values[0]))
                      & (most_points['season_90count'] > 24))
                   |(most_points['overall_90count'] > 38))]['name'].drop_duplicates()) | 
                             
                             
                             (most_points.name.isin(last_4gws_min.name)) | (most_points.name.isin(player_inclusions))) 
                              ]

    most_points = most_points.groupby(['new_position', 'name', 'fixture_difficulty']).agg({
                                                                    'total_points': 'sum', 
                                                                    'total_points_adj': 'sum', 
                                                                    'minutes': 'sum', 
                                                                    'minutes_adj': 'sum', 
                                                                    'bonus': 'sum',
                                                                    'goals_scored': 'sum',
                                                                    'assists': 'sum',                                                        
                                                                    'clean_sheets': 'sum', 
                                                                    'now_cost': 'max'
                                                                    }).sort_values('total_points_adj', ascending = False).reset_index()

   
    
    if season_not_started:
        current_season_string = str(int(current_season) + 1) + '-' + str(int(current_season) + 2)[-2:]
    else:
        current_season_string = current_season + '-' + str(int(current_season) + 1)[-2:]
        
    fixtures_current_season = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + current_season_string + '/fixtures.csv', encoding='latin-1')
    teams_current_season = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + current_season_string + '/teams.csv', encoding='latin-1')
    players_current_season = pd.read_csv('C:/Users/mvellera/OneDrive - ascenaretail.com/Documents/GitHub/Fantasy-Premier-League/data/' + current_season_string + '/players_raw.csv', encoding='latin-1')
    current_player_team_mapping = pd.merge(players_current_season, teams_current_season, 'left', left_on = 'team', right_on ='id')
    current_player_team_mapping['name'] = current_player_team_mapping['first_name'] +  ' ' + current_player_team_mapping['second_name']
    current_player_team_mapping = current_player_team_mapping.rename(columns = {'short_name': 'team_name'})
    
    most_points = pd.merge(most_points, current_player_team_mapping[['name', 'team_name']].drop_duplicates(), on = 'name')

    most_points['ppm90'] = ((most_points.total_points_adj/most_points.minutes_adj)*90)
    most_points['ppm90_orig'] = ((most_points.total_points/most_points.minutes)*90)
    most_points['90count'] = (most_points['minutes']/90).round().astype('int')
    most_points['bonus90'] = ((most_points['bonus']/most_points.minutes)*90).round(1)
    most_points['now_cost'] = most_points['now_cost']/10
    
    ppm = most_points.sort_values('ppm90', ascending=False).reset_index(drop = True)
    ppm = ppm[ppm['90count'] > 1]    
    

    fixtures_current_season['kickoff_time'] = pd.to_datetime(fixtures_current_season['kickoff_time'])
    fixtures_current_season = fixtures_current_season[(fixtures_current_season['kickoff_time'] > last_gw_kickoff) 
                                                      & ~(fixtures_current_season['kickoff_time'].isnull()) 
                                                      & (fixtures_current_season['event'] <= max_of_next_gws)]
    fixtures_current_season = fixtures_current_season[['kickoff_time', 'team_a', 'team_a_difficulty', 'team_h', 'team_h_difficulty']]

    fixtures_current_season = pd.merge(fixtures_current_season, teams_current_season[['id', 'short_name']], left_on = 'team_a', right_on = 'id')
    fixtures_current_season = pd.merge(fixtures_current_season, teams_current_season[['id', 'short_name']], left_on = 'team_h', right_on = 'id')

    fixtures_current_season.rename(columns={'short_name_x': 'team_a_name', 
                         'short_name_y': 'team_h_name'}, inplace = True)
    fixtures_current_season.drop(['id_x', 'id_y'], axis = 1, inplace = True)
    fixtures_current_season = fixtures_current_season.sort_values('kickoff_time')

    #Get Current Teams
    current_teams = teams_current_season.short_name.unique()  
    if lookahead == True:
        lookahead = 1
    else:
        lookahead = next_fixture_count
        
    if skip_next_gw == True:
        skipped_next_fixture_count = next_fixture_count - 1
    else:
        skipped_next_fixture_count = next_fixture_count

    next_fixtures = pd.DataFrame()

    for team in current_teams:
        fixtures = fixtures_current_season[(fixtures_current_season['team_a_name'] == team) 
                                           | (fixtures_current_season['team_h_name'] == team)]\
                   .head(next_fixture_count).tail(skipped_next_fixture_count).tail(lookahead)
        fixtures.loc[:,'was_home'] = np.where(fixtures['team_h_name'] == team, True, False)
        fixtures['opposition_was_home'] = ~fixtures['was_home']
        fixtures['team_name'] = np.where(fixtures['was_home'] == True, fixtures['team_h_name'], fixtures['team_a_name'])
        fixtures['opponent_team_name'] =  np.where(fixtures['was_home'] == True, fixtures['team_a_name'], fixtures['team_h_name'])

        fixtures = fixtures[['team_name', 'opponent_team_name', 'was_home', 'opposition_was_home']]
        fixtures = pd.merge(fixtures, fixture_difficulty[['team_name', 'was_home', 'fixture_difficulty']], left_on = ['opponent_team_name', 'opposition_was_home'], right_on = ['team_name', 'was_home'])

        fixtures.drop(['team_name_y', 'was_home_y'], axis = 1, inplace = True)
        fixtures.rename(columns={'team_name_x': 'team_name', 'was_home_x': 'was_home'}, inplace = True)

        next_fixtures = next_fixtures.append(fixtures)

    ppm_next_fixtures = pd.merge(next_fixtures, ppm, on = ['team_name', 'fixture_difficulty'])
    
    ppm = ppm_next_fixtures.groupby(['new_position', 'team_name', 'name']).agg({
                                                                'total_points': 'sum', 
                                                                'total_points_adj': 'sum', 
                                                                'minutes': 'sum', 
                                                                'minutes_adj': 'sum', 
                                                                'bonus': 'mean',
                                                                'goals_scored': 'mean',
                                                                'assists': 'mean',                                                              
                                                                'clean_sheets': 'mean', 
                                                                'ppm90': 'mean',
                                                                '90count': 'mean',
                                                                'now_cost': 'max'
                                                                }).sort_values('total_points_adj', ascending = False).reset_index()

    ppm['ppm90'] = ((ppm.total_points_adj/ppm.minutes_adj)*90)
    ppm = ppm.sort_values('ppm90', ascending = False).reset_index()
    ppm = ppm[ppm.total_points_adj > 2].reset_index(drop = True)
    return ppm, next_fixtures

In [11]:
from pulp import *
import re
import copy
def getOptimalTeam(ppm, metric, def_cnt, mid_cnt, fwd_cnt, player_exclusions):

    ppm = ppm[~(ppm.name.isin(player_exclusions))].reset_index(drop = True)
    formation = str(def_cnt) + str(mid_cnt) + str(fwd_cnt)
    problem_str = 'FantasyTeam' + str(def_cnt) + str(mid_cnt) + str(fwd_cnt) + str(metric)
    prob = pulp.LpProblem(problem_str, pulp.LpMaximize)
    decision_variables = []
    for rownum, row in ppm.iterrows():
        variable = str('x' + str(rownum))
        variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
        decision_variables.append(variable)

#    print ("Total number of decision_variables: " + str(len(decision_variables)))

    optim_fn = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                formula = row[metric]*player
                optim_fn += formula

    prob += optim_fn
    
    if((formation == '532') | (formation == '541')):
        #avail_cash = 81.4
        avail_cash = 82.5
        #avail_cash = 82.4
        #avail_cash = 81.9
    else:
        #avail_cash = 81.1
        avail_cash = 83.
        #avail_cash = 82.9
        #avail_cash = 82.4
    total_paid = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                formula = row['now_cost']*player
                total_paid += formula

    prob += (total_paid <= avail_cash)

    avail_gk = 1
    total_gk = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['new_position'] == 'GK':
                    formula = 1*player
                    total_gk += formula
    prob += (total_gk == avail_gk)

    total_def = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['new_position'] == 'DEF':
                    formula = 1*player
                    total_def += formula
    prob += total_def == def_cnt

    total_mid = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['new_position'] == 'MID':
                    formula = 1*player
                    total_mid += formula
    prob += total_mid == mid_cnt

    total_fwd = ""
    for rownum, row in ppm.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['new_position'] == 'FWD':
                    formula = 1*player
                    total_fwd += formula
    prob += total_fwd == fwd_cnt

    team_dict= {}
    for team in set(ppm.team_name):
        team_dict[str(team)]=dict()
        team_dict[str(team)]['avail'] = 3
        team_dict[str(team)]['total'] = ""
        for rownum, row in ppm.iterrows():
            for i, player in enumerate(decision_variables):
                if rownum == i:
                    if row['team_name'] == team:
                        formula = 1*player
                        team_dict[str(team)]['total'] += formula

        prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])
 #   print(prob)

    prob.writeLP(problem_str + '.lp')
    optimization_result = prob.solve()
    assert optimization_result == pulp.LpStatusOptimal

    variable_name = []
    variable_value = []

    for v in prob.variables():
        variable_name.append(v.name)
        variable_value.append(v.varValue)

    df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
    for rownum, row in df.iterrows():
        value = re.findall(r'(\d+)', row['variable'])
        df.loc[rownum, 'variable'] = int(value[0])

    df = df.sort_index(by='variable')

    #append results
    for rownum, row in ppm.iterrows():
        for results_rownum, results_row in df.iterrows():
            if rownum == results_row['variable']:
                ppm.loc[rownum, 'decision'] = results_row['value']

    return (ppm, ppm[ppm.decision==1][metric].sum())

In [12]:
# iterations = pd.DataFrame(columns = ['player_points_decay', 'fixture_difficulty_decay', 'fixture_clusters', 'max_gw', 'score'])
# for player_points_decay in np.linspace(0.015, 0.017, 3):
#     for fixture_difficulty_decay in np.linspace(0.007, 0.008, 2):
#         for fixture_clusters in [7]:
#             for max_gw in [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
#                 ppm, next_fixtures = getPPM(player_points_decay, fixture_difficulty_decay, fixture_clusters, max_gw, 1)
#                 player_exclusions = []
                
#                 while True:
#                     ppm442, result442 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 4, mid_cnt = 4, fwd_cnt = 2,
#                                                        player_exclusions = player_exclusions)
#                     ppm352, result352 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 3, mid_cnt = 5, fwd_cnt = 2,
#                                                        player_exclusions = player_exclusions)
#                     ppm532, result532 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 5, mid_cnt = 3, fwd_cnt = 2,
#                                                        player_exclusions = player_exclusions)
#                     ppm343, result343 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 3, mid_cnt = 4, fwd_cnt = 3,
#                                                        player_exclusions = player_exclusions)
#                     ppm433, result433 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 4, mid_cnt = 3, fwd_cnt = 3,
#                                                        player_exclusions = player_exclusions)
#                     ppm451, result451 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 4, mid_cnt = 5, fwd_cnt = 1,
#                                                        player_exclusions = player_exclusions)
#                     ppm541, result541 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', 
#                                                        def_cnt = 5, mid_cnt = 4, fwd_cnt = 1,
#                                                        player_exclusions = player_exclusions)

#                     ppm_scores = pd.DataFrame()
#                     ppm_scores['formation'] = ['442', '352', '532', '343', '433', '451', '541']
#                     ppm_scores['scores'] = [result442, result352, result532, result343, result433, result451, result541]
#                     best_formation = ppm_scores.sort_values('scores', ascending = False).head(1).formation.values[0]
#                     if best_formation == '442':
#                         best_ppm = ppm442[ppm442.decision==1]
#                     if best_formation == '352':
#                         best_ppm = ppm352[ppm352.decision==1]
#                     if best_formation == '532':
#                         best_ppm = ppm532[ppm532.decision==1]
#                     if best_formation == '343':
#                         best_ppm = ppm343[ppm343.decision==1]
#                     if best_formation == '433':
#                         best_ppm = ppm433[ppm433.decision==1]
#                     if best_formation == '451':
#                         best_ppm = ppm451[ppm451.decision==1]
#                     if best_formation == '541':
#                         best_ppm = ppm541[ppm541.decision==1]   

#                     actuals = pd.merge(best_ppm, 
#                                        gw_original[(gw_original.season.str.split('-').str[0] == current_season) & (gw_original['gw'] == max_gw + 1)][['name', 'total_points', 'minutes']], 
#                                        on = 'name',
#                                        how = 'left')
#                     minutes0_players = list(actuals[actuals['minutes_y'].isnull()].name.unique())
#                     if(len(minutes0_players) == 0):
#                         break
#                     player_exclusions = player_exclusions + minutes0_players
#                     print("GW: " + str(max_gw + 1) + "\nPlayer Exclusions: " + str(player_exclusions))

#                 iterations = iterations.append({'player_points_decay': player_points_decay, 
#                                                 'fixture_difficulty_decay': fixture_difficulty_decay,
#                                                 'fixture_clusters': fixture_clusters,
#                                                 'max_gw': max_gw,
#                                                 'score': actuals.total_points_y.sum()}, ignore_index=True)

#                 iter_len = len(iterations)
#                 print(iter_len)
                
#                 if iter_len % 10 == 0:
#                     try:
#                         iterations.to_csv("Iterations_7Clusters_Remaining.csv", index=False)
#                     except:
#                         pass

In [13]:
gw[gw.kickoff_time == max(gw.kickoff_time)].head(1)

,kickoff_time,season,gw,name,team_name,opponent_team_name,team_h_score,team_a_score,new_position,total_points,minutes,bonus,goals_scored,goals_conceded,assists,was_home,yellow_cards,red_cards,saves,threat,ict_index,clean_sheets,now_cost,code
4032,2020-12-30 20:00:00,2020-21,16,Jordan Henderson,LIV,NEW,0.0,0.0,MID,3,90,0,0,0,0,False,0,0,0,1.0,3.1,1,54.0,56979.0


In [14]:
gw[gw.name.str.contains('Ziyech')].tail()

,kickoff_time,season,gw,name,team_name,opponent_team_name,team_h_score,team_a_score,new_position,total_points,minutes,bonus,goals_scored,goals_conceded,assists,was_home,yellow_cards,red_cards,saves,threat,ict_index,clean_sheets,now_cost,code
7920,2020-10-31 15:00:00,2020-21,7,Hakim Ziyech,CHE,BUR,0.0,3.0,MID,14,72,3,1,0,1,False,0,0,0,11.0,7.9,1,81.0,124183.0
8520,2020-11-07 17:30:00,2020-21,8,Hakim Ziyech,CHE,SHU,4.0,1.0,MID,11,90,3,0,1,2,True,0,0,0,23.0,15.2,0,81.0,124183.0
9121,2020-11-21 12:30:00,2020-21,9,Hakim Ziyech,CHE,NEW,0.0,2.0,MID,3,86,0,0,0,0,False,0,0,0,3.0,5.0,1,81.0,124183.0
1024,2020-11-29 16:30:00,2020-21,10,Hakim Ziyech,CHE,TOT,0.0,0.0,MID,2,82,0,0,0,0,True,1,0,0,13.0,4.9,1,81.0,124183.0
1565,2020-12-05 20:00:00,2020-21,11,Hakim Ziyech,CHE,LEE,3.0,1.0,MID,1,29,0,0,1,0,True,0,0,0,19.0,2.9,0,81.0,124183.0


In [15]:
print(gw[gw.kickoff_time == max(gw.kickoff_time)].gw.max())

16


In [16]:
gw.loc[gw.name.str.contains('Aleksandar Mitrovic'), 'name'] = 'Aleksandar MitroviÄ'

In [17]:
gw[gw.name.str.contains('Aleksandar MitroviÄ')].tail()

,kickoff_time,season,gw,name,team_name,opponent_team_name,team_h_score,team_a_score,new_position,total_points,minutes,bonus,goals_scored,goals_conceded,assists,was_home,yellow_cards,red_cards,saves,threat,ict_index,clean_sheets,now_cost,code
8931,2020-11-22 12:00:00,2020-21,9,Aleksandar MitroviÄ,FUL,EVE,2.0,3.0,FWD,1,32,0,0,0,0,True,0,0,0,29.0,4.5,0,55.0,128389.0
833,2020-11-30 17:30:00,2020-21,10,Aleksandar MitroviÄ,FUL,LEI,1.0,2.0,FWD,1,3,0,0,0,0,False,0,0,0,0.0,1.5,0,55.0,128389.0
2587,2020-12-16 20:00:00,2020-21,13,Aleksandar MitroviÄ,FUL,BHA,0.0,0.0,FWD,1,13,0,0,0,0,True,0,0,0,6.0,1.8,0,55.0,128389.0
3197,2020-12-19 20:00:00,2020-21,14,Aleksandar MitroviÄ,FUL,NEW,1.0,1.0,FWD,2,76,0,0,1,0,False,0,0,0,17.0,2.5,0,55.0,128389.0
3811,2020-12-26 15:00:00,2020-21,15,Aleksandar MitroviÄ,FUL,SOU,0.0,0.0,FWD,1,7,0,0,0,0,True,0,0,0,6.0,0.5,0,55.0,128389.0


In [18]:
max_gw = gw[gw.kickoff_time == max(gw.kickoff_time)].gw.max()
next_fixture_count = 1
player_inclusions = []   

ppm, next_fixtures = getPPM(player_points_decay = 0.007, 
                            fixture_difficulty_decay = 0.003, 
                            fixture_clusters = 6, 
                            max_gw = max_gw, 
                            next_fixture_count = next_fixture_count, 
                            season_not_started = False,
                            max_of_next_gws = max_gw + next_fixture_count,
                            lookahead = False,
                            skip_next_gw = False,
                            player_inclusions = player_inclusions)   

current_season = str(gw.season.str.split('-').str[0].astype('int').drop_duplicates().nlargest()[0:1].values[0])

player_exclusions = ['Riyad Mahrez', 'Tammy Abraham']


ppm.loc[ppm.name.str.contains('Fernandes'), 'now_cost'] = 10.5
ppm.loc[ppm.name.str.contains('Salah'), 'now_cost'] = 12.2
ppm.loc[ppm.name.str.contains('Calvert'), 'now_cost'] = 7.3
ppm.loc[ppm.name.str.contains('Mitchell'), 'now_cost'] = 4.0
ppm.loc[ppm.name.str.contains('Heung'), 'now_cost'] = 9.4
ppm.loc[ppm.name.str.contains('Bruyne'), 'now_cost'] = 11.5
ppm.loc[ppm.name.str.contains('Bamford'), 'now_cost'] = 6.1
ppm.loc[ppm.name.str.contains('Wilson'), 'now_cost'] = 6.5

ppm442, result442 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 4, mid_cnt = 4, fwd_cnt = 2, player_exclusions = player_exclusions)
ppm352, result352 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 3, mid_cnt = 5, fwd_cnt = 2, player_exclusions = player_exclusions)
ppm532, result532 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 5, mid_cnt = 3, fwd_cnt = 2, player_exclusions = player_exclusions)
ppm343, result343 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 3, mid_cnt = 4, fwd_cnt = 3, player_exclusions = player_exclusions)
ppm433, result433 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 4, mid_cnt = 3, fwd_cnt = 3, player_exclusions = player_exclusions)
ppm451, result451 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 4, mid_cnt = 5, fwd_cnt = 1, player_exclusions = player_exclusions)
ppm541, result541 = getOptimalTeam(copy.deepcopy(ppm), 'ppm90', def_cnt = 5, mid_cnt = 4, fwd_cnt = 1, player_exclusions = player_exclusions)

ppm_scores = pd.DataFrame()
ppm_scores['formation'] = ['442', '352', '532', '343', '433', '451', '541']
ppm_scores['scores'] = [result442, result352, result532, result343, result433, result451, result541]
best_formation = ppm_scores.sort_values('scores', ascending = False).head(1).formation.values[0]
if best_formation == '442':
    best_ppm = ppm442[ppm442.decision==1]
if best_formation == '352':
    best_ppm = ppm352[ppm352.decision==1]
if best_formation == '532':
    best_ppm = ppm532[ppm532.decision==1]
if best_formation == '343':
    best_ppm = ppm343[ppm343.decision==1]
if best_formation == '433':
    best_ppm = ppm433[ppm433.decision==1]
if best_formation == '451':
    best_ppm = ppm451[ppm451.decision==1]
if best_formation == '541':
    best_ppm = ppm541[ppm541.decision==1]   

# actuals = pd.merge(best_ppm, gw_original[(gw_original.season.str.split('-').str[0] == current_season) & (gw_original['gw'] == max_gw + 1)][['name', 'total_points']], on = 'name')


    season team_name new_position                              name  \
0  2020-21       MUN          MID     Bruno Miguel Borges Fernandes   
1  2020-21       TOT          MID                     Heung-Min Son   
2  2020-21       LIV          MID                     Mohamed Salah   
3  2020-21       TOT          FWD                        Harry Kane   
4  2020-21       MCI          MID                   Kevin De Bruyne   
5  2020-21       EVE          FWD             Dominic Calvert-Lewin   
6  2019-20       LEI          DEF  Ricardo Domingos Barbosa Pereira   
7  2020-21       TOT          MID                     Heung-Min Son   
8  2020-21       LEI          FWD                       Jamie Vardy   
9  2020-21       MUN          MID                   Marcus Rashford   

   fixture_difficulty  total_points  total_points_adj  minutes  minutes_adj  \
0                   1          45.0         39.164341    315.0   277.535288   
1                   2          40.0         32.113418    430

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [19]:
next_fixtures

,team_name,opponent_team_name,was_home,opposition_was_home,fixture_difficulty
0,ARS,WBA,False,True,3
0,AVL,MUN,False,True,1
0,BHA,WOL,True,False,0
0,BUR,FUL,True,False,0
0,CHE,MCI,True,False,2
0,CRY,SHU,True,False,0
0,EVE,WHU,True,False,2
0,FUL,BUR,False,True,3
0,LEI,NEW,False,True,3
0,LEE,TOT,False,True,1


In [20]:
# ppm343[(ppm343['now_cost'] <= 4.5) & (ppm343['new_position'] == 'DEF')]
# ppm343[(ppm343['now_cost'] <= 5.5) & (ppm343['new_position'] == 'MID')]

In [21]:
ppm343[ppm343['new_position'].isin(['GK', 'DEF'])].groupby(['team_name']).median()['ppm90'].sort_values(ascending=False)

team_name
SOU    7.940616
ARS    5.847812
BUR    5.476848
LEE    4.796404
AVL    4.754157
CHE    4.592794
WHU    4.191107
NEW    3.937093
TOT    3.811879
BHA    3.795765
WOL    3.265002
LIV    3.244016
CRY    3.128613
MUN    2.850166
LEI    2.716540
EVE    2.629427
FUL    2.500000
SHU    2.045173
WBA    1.468542
Name: ppm90, dtype: float64

In [22]:
ppm343

,index,new_position,team_name,name,total_points,total_points_adj,minutes,minutes_adj,bonus,goals_scored,assists,clean_sheets,ppm90,90count,now_cost,decision
0,49,MID,AVL,Anwar El Ghazi,27.0,10.817128,441.0,77.147095,1.0,1.0,2.0,0.0,12.619289,5,5.8,1.0
1,61,MID,SOU,James Ward-Prowse,42.0,9.499625,868.0,77.849069,4.0,2.0,2.0,2.0,10.982357,10,6.3,1.0
2,11,DEF,CHE,Thiago Thiago,20.0,19.304289,180.0,170.401407,2.0,1.0,0.0,2.0,10.195843,2,5.6,1.0
3,43,FWD,ARS,Alexandre Lacazette,99.0,11.730614,1746.0,105.499145,14.0,7.0,6.0,4.0,10.007240,19,8.2,1.0
4,18,MID,LEE,Raphael Dias Belloli,18.0,16.333186,181.0,165.754923,1.0,1.0,2.0,1.0,8.868435,2,5.5,1.0
5,17,FWD,LEI,Jamie Vardy,155.0,16.667214,2571.0,180.732600,29.0,17.0,2.0,8.0,8.299827,29,10.2,1.0
6,95,DEF,SOU,Jan Bednarek,17.0,6.901223,675.0,78.219378,1.0,2.0,0.0,0.0,7.940616,8,4.8,1.0
7,12,DEF,EVE,Yerry Mina,37.0,18.572795,636.0,211.873026,4.0,2.0,0.0,2.0,7.889403,7,5.5,1.0
8,26,MID,LIV,Mohamed Salah,159.0,14.909411,2223.0,170.415240,11.0,14.0,6.0,9.0,7.873985,25,12.2,1.0
9,173,MID,MCI,Kevin De Bruyne,27.0,2.130402,528.0,24.459219,5.0,2.0,0.0,1.0,7.839014,6,11.5,0.0


In [23]:
best_ppm

,index,new_position,team_name,name,total_points,total_points_adj,minutes,minutes_adj,bonus,goals_scored,assists,clean_sheets,ppm90,90count,now_cost,decision
0,49,MID,AVL,Anwar El Ghazi,27.0,10.817128,441.0,77.147095,1.0,1.0,2.0,0.0,12.619289,5,5.8,1.0
1,61,MID,SOU,James Ward-Prowse,42.0,9.499625,868.0,77.849069,4.0,2.0,2.0,2.0,10.982357,10,6.3,1.0
2,11,DEF,CHE,Thiago Thiago,20.0,19.304289,180.0,170.401407,2.0,1.0,0.0,2.0,10.195843,2,5.6,1.0
3,43,FWD,ARS,Alexandre Lacazette,99.0,11.730614,1746.0,105.499145,14.0,7.0,6.0,4.0,10.007240,19,8.2,1.0
4,18,MID,LEE,Raphael Dias Belloli,18.0,16.333186,181.0,165.754923,1.0,1.0,2.0,1.0,8.868435,2,5.5,1.0
5,17,FWD,LEI,Jamie Vardy,155.0,16.667214,2571.0,180.732600,29.0,17.0,2.0,8.0,8.299827,29,10.2,1.0
6,95,DEF,SOU,Jan Bednarek,17.0,6.901223,675.0,78.219378,1.0,2.0,0.0,0.0,7.940616,8,4.8,1.0
7,12,DEF,EVE,Yerry Mina,37.0,18.572795,636.0,211.873026,4.0,2.0,0.0,2.0,7.889403,7,5.5,1.0
8,26,MID,LIV,Mohamed Salah,159.0,14.909411,2223.0,170.415240,11.0,14.0,6.0,9.0,7.873985,25,12.2,1.0
9,173,MID,MCI,Kevin De Bruyne,27.0,2.130402,528.0,24.459219,5.0,2.0,0.0,1.0,7.839014,6,11.5,1.0


In [24]:
best_ppm.sum()['now_cost']

80.19999999999999

In [25]:
ppm343[ppm343.decision==1]

,index,new_position,team_name,name,total_points,total_points_adj,minutes,minutes_adj,bonus,goals_scored,assists,clean_sheets,ppm90,90count,now_cost,decision
0,49,MID,AVL,Anwar El Ghazi,27.0,10.817128,441.0,77.147095,1.0,1.0,2.0,0.0,12.619289,5,5.8,1.0
1,61,MID,SOU,James Ward-Prowse,42.0,9.499625,868.0,77.849069,4.0,2.0,2.0,2.0,10.982357,10,6.3,1.0
2,11,DEF,CHE,Thiago Thiago,20.0,19.304289,180.0,170.401407,2.0,1.0,0.0,2.0,10.195843,2,5.6,1.0
3,43,FWD,ARS,Alexandre Lacazette,99.0,11.730614,1746.0,105.499145,14.0,7.0,6.0,4.0,10.007240,19,8.2,1.0
4,18,MID,LEE,Raphael Dias Belloli,18.0,16.333186,181.0,165.754923,1.0,1.0,2.0,1.0,8.868435,2,5.5,1.0
5,17,FWD,LEI,Jamie Vardy,155.0,16.667214,2571.0,180.732600,29.0,17.0,2.0,8.0,8.299827,29,10.2,1.0
6,95,DEF,SOU,Jan Bednarek,17.0,6.901223,675.0,78.219378,1.0,2.0,0.0,0.0,7.940616,8,4.8,1.0
7,12,DEF,EVE,Yerry Mina,37.0,18.572795,636.0,211.873026,4.0,2.0,0.0,2.0,7.889403,7,5.5,1.0
8,26,MID,LIV,Mohamed Salah,159.0,14.909411,2223.0,170.415240,11.0,14.0,6.0,9.0,7.873985,25,12.2,1.0
11,42,FWD,MUN,Anthony Martial,93.0,11.918221,1359.0,153.296795,8.0,7.0,7.0,5.0,6.997145,15,8.7,1.0
